# The Best Pittsburgh Neighborhood

## Combines the Results from Sidewalk_Ratio, City_Of_Pittsburgh_Traffic_Count, and Final-Stairs Notebooks to Determine the Best Neighborhood Overall for Pedestrians

In [1]:
import pandas as pd

In [13]:
%store -r sorted_ratios
sorted_ratios.head(10)

,Ratios
Bedford Dwellings,6.644908
Allegheny Center,3.227465
Terrace Village,2.732608
Northview Heights,2.712593
North Oakland,2.196612
North Shore,2.174160
Point Breeze North,2.064611
Squirrel Hill North,2.024627
Central Lawrenceville,2.008441
South Side Flats,1.955125


In [11]:
%store -r final_df
final_df.head(10)

,neighborhood,total_stairs/counts
63,Allegheny West,7.666667
77,Mount Oliver Borough,9.000000
60,Esplen,16.000000
59,North Oakland,18.000000
49,Allegheny Center,23.333333
65,Shadyside,25.000000
64,East Carnegie,26.000000
66,St. Clair,26.500000
38,Knoxville,26.750000
69,South Shore,27.500000
